<h1>Description of the Problem and Discussion of the Background</h1>

<h2>Deciding where to open a new venue in London (outside Inner City Limits)</h2>

London is a great city. It is diverse, multicultural and full of opportunities. But there is also another side to its popularity and appeal. London is an expensive place to live and thrive. Many businesses want to open a venue here and many pay top price to have their window on Piccadilly or Oxford Street. But what about those, who want to start their own business and cannot really afford to open in the City yet? Where is it best to open a new place? Where will it be cheapest and will have enough people living around to be popular? Where the competition is not too overwhelming?

All these questions and more will be answered below. The whole process will be carried out via the below plan:

<font color=deepskyblue> Obtain the Data </font>

1.a. Name of the London Boroughs, area and population from web scrapping

1.b. Filter the boroughs to get the best candidates for the analysis.

1.c. Use Foresquare Data to obtain info about most popular venues.

<font color=deepskyblue> Data Visualization and Some Simple Statistical Analysis. </font>

<font color=deepskyblue> Analysis Using Clustering, Specially K-Means Clustering. </font>

3.a. Maximize the number of clusters.

3.b. Visualization using Chloropleth Map

<font color=deepskyblue> Compare the Neighborhoods to Find the Best Place for Starting up a Venue. </font> <font color=deepskyblue> Inference From these Results and related Conclusions. </font>

<font color=orangered>Target Audience</font>

First time enterpreneurs, who want to start their first business. Below dataset will give a comprehensive insight into where best to open a new venue, to maximise the value for money.

People who already run a business and want to branch out. Given the extra information, it may provide valuable information before decision making.

First, we get the nessessary information on London Boroughs, dropping the extras, that will not be needed for the analysis.

In [24]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

url='https://en.wikipedia.org/wiki/List_of_London_boroughs'
LDF = pd.read_html(url, header=0)[0]
LDF.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [6]:
LF = LDF.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
LF['Inner'].replace(np.nan, 0, inplace=True)
LF['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
LF['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
LF['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
LF['Inner'] = '0'
LF.head()

,Borough,Inner,Area (sq mi),Population (2013 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
1,Barnet,0,33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


Then we rename the columns, making the dataset better on the eyes. Because of extra notes in the Wiki page, we will rename some of the Boroughs. Due to the staggering difference in rent price, as well as the ammount of venues in London, we will filter to have only the Outer boroughs going forward

In [7]:
LF['Inner'] = LF.Borough.isin(Inn).astype(int)
Out = LF[LF.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
df = Out.rename(columns = {"Area (sq mi)": "Area", "Population (2013 est)[1]":"Population"})
geolocator = Nominatim(user_agent = 'London_explorer')
df['Co-ordinates']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Co-ordinates'].apply(pd.Series)
df.head()

,Borough,Area,Population,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,194352,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,369088,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,236687,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,317264,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,317899,"(51.4028046, 0.0148142)",51.402805,0.014814


Finally, we edit the coordinates, that are incorrect. Lets filter the set even more - because rent price is a big issue for the new business, we will get the boroughs with the lowest ones.

In [8]:
Max_Rent = ['102.25','150.75','97','150.75','118.5','129.25','140','102.25','107.75','140','86','161.5','161.5','140','123.75','134.5','118.5','140','129.25','145.25']
df['Max_Rent'] = Max_Rent
df["Max_Rent"] = pd.to_numeric(df["Max_Rent"])
fin = df[df.Max_Rent <= 125]
Long_list = fin['Longitude'].tolist()
Lat_list = fin['Latitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)
replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}
longtitudes_new = [replace_longitudes.get(n7,n7) for n7 in Long_list]
latitudes_new = [replace_latitudes.get(n7,n7) for n7 in Lat_list]
fin = fin.drop('Co-ordinates', axis=1)
fin['Longitude'] = longtitudes_new
fin['Latitude'] = latitudes_new
fin

Old latitude list:  [51.5541171, 39.9692378, 51.4028046, 51.6520851, 51.587929849999995, 51.0043613, 51.410802849999996, 51.5763203]
Old Longitude list:  [0.15050434261994267, -82.936864, 0.0148142, -0.0810175, -0.10541010599099046, -2.337474942629507, -0.18809850595572736, 0.0454097]


,Borough,Area,Population,Latitude,Longitude,Max_Rent
0,Barking and Dagenham,13.93,194352,51.554117,0.150504,102.25
2,Bexley,23.38,236687,39.969238,-82.936864,97.00
4,Bromley,57.97,317899,51.402805,0.014814,118.50
8,Enfield,31.74,320524,51.652085,-0.081018,102.25
12,Haringey,11.42,263386,51.587930,-0.105410,107.75
14,Havering,43.35,242080,51.004361,-2.337475,86.00
22,Merton,14.52,203223,51.410803,-0.188099,123.75
24,Redbridge,21.78,288272,51.576320,0.045410,118.50


In [9]:
location = geolocator.geocode('London')
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [10]:
!pip install folium
import folium
Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)
for lat, lng, label in zip(fin['Latitude'], fin['Longitude'], fin['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=9, popup=label, color='Red', fill=True, fill_color='#Blue', fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh

     |████████████████████████████████| 102kB 10.6MB/s ta 0:00:01


In [11]:
CLIENT_ID = 'QPT2O4JOXLDJW43COYX2F2N5XS02ATEO5W4POCBNGJ4WPS3Y' 
CLIENT_SECRET = 'NHZXDY231F3WQ2WRJOMRCM1B34YPFWHGVG454INTLYNX05C5' 
VERSION = '20190915' 
print('My credentails:')
print('My CLIENT_ID: ' + CLIENT_ID)
print('My CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
My CLIENT_ID: QPT2O4JOXLDJW43COYX2F2N5XS02ATEO5W4POCBNGJ4WPS3Y
My CLIENT_SECRET:NHZXDY231F3WQ2WRJOMRCM1B34YPFWHGVG454INTLYNX05C5


In [14]:
radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name, lat, lng,
                            v['venue']['name'],
                            v['venue']['location']['lat'],
                            v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 'Latitude', 'Longitude', 'Venue', 'Venue_Lat', 'Venue_Long', 'Venue_Category']
    return(nearby_venues)

In [15]:
Brgh_Venues = getVenues(names=fin['Borough'], latitudes=fin['Latitude'], longitudes=fin['Longitude'])

Lets count the venues in every borough, to get the data scale

In [16]:
Brgh_Venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
Borough,,,,,,
Barking and Dagenham,98,98,98,98,98,98
Bexley,89,89,89,89,89,89
Bromley,100,100,100,100,100,100
Enfield,100,100,100,100,100,100
Haringey,100,100,100,100,100,100
Havering,6,6,6,6,6,6
Merton,100,100,100,100,100,100
Redbridge,100,100,100,100,100,100


Lets break these numbers down to see which venues are more popular than the others

In [18]:
London_Brgh_onehot = pd.get_dummies(Brgh_Venues[['Venue_Category']], prefix="", prefix_sep="")
mid = Brgh_Venues['Borough']
London_Brgh_onehot.insert(0, 'Borough', mid)
London_Brgh_onehot.head()

,Borough,ATM,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
Brgh_grouped = London_Brgh_onehot.groupby('Borough').mean().reset_index()
Brgh_grouped

,Borough,ATM,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Barking and Dagenham,0.000000,0.010204,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.010204,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
1,Bexley,0.011236,0.011236,0.00,0.00,0.011236,0.00,0.00,0.011236,0.011236,...,0.00,0.000000,0.011236,0.00,0.022472,0.011236,0.00,0.00,0.011236,0.011236
2,Bromley,0.000000,0.010000,0.00,0.00,0.000000,0.01,0.01,0.000000,0.000000,...,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
3,Enfield,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.01,0.000000,0.000000,...,0.00,0.010000,0.070000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
4,Haringey,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.04,0.000000,0.080000,0.01,0.000000,0.000000,0.00,0.01,0.000000,0.000000
5,Havering,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
6,Merton,0.000000,0.000000,0.01,0.00,0.000000,0.02,0.00,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.01,0.000000,0.000000
7,Redbridge,0.000000,0.000000,0.00,0.02,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.000000,0.010000,0.01,0.000000,0.000000,0.00,0.01,0.000000,0.000000


With all those zeros, it is easy to assume, that people there just sit at home! Lets see, which of the venues are most popular?

In [20]:
num_top_venues = 5
for brgh in Brgh_grouped['Borough']:
    print("_________"+brgh+"________")
    temp = Brgh_grouped[Brgh_grouped['Borough'] == brgh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_________Barking and Dagenham________
           venue  freq
0    Supermarket  0.09
1           Park  0.09
2  Grocery Store  0.07
3            Pub  0.07
4    Coffee Shop  0.07


_________Bexley________
            venue  freq
0     Coffee Shop  0.06
1     Pizza Place  0.06
2  Ice Cream Shop  0.04
3  Discount Store  0.04
4  Sandwich Place  0.03


_________Bromley________
           venue  freq
0    Coffee Shop  0.09
1            Pub  0.08
2  Grocery Store  0.08
3           Park  0.07
4    Pizza Place  0.05


_________Enfield________
                venue  freq
0         Coffee Shop  0.09
1  Turkish Restaurant  0.07
2                 Pub  0.07
3       Garden Center  0.05
4    Greek Restaurant  0.05


_________Haringey________
                venue  freq
0                Café  0.11
1                 Pub  0.11
2                Park  0.08
3  Turkish Restaurant  0.08
4              Bakery  0.04


_________Havering________
               venue  freq
0   Indoor Play Area  0.17
1        IT Serv

Convert NumPy Arrays to dataframe

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
brgh_venues_sorted = pd.DataFrame(columns = columns)
brgh_venues_sorted['Borough'] = Brgh_grouped['Borough']
for ind in np.arange(Brgh_grouped.shape[0]):
    brgh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brgh_grouped.iloc[ind, :], num_top_venues)
brgh_venues_sorted.head(8)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,Park,Supermarket,Coffee Shop,Grocery Store,Pub
1,Bexley,Pizza Place,Coffee Shop,Ice Cream Shop,Discount Store,Restaurant
2,Bromley,Coffee Shop,Grocery Store,Pub,Park,Gym / Fitness Center
3,Enfield,Coffee Shop,Pub,Turkish Restaurant,Café,Park
4,Haringey,Pub,Café,Turkish Restaurant,Park,Coffee Shop
5,Havering,IT Services,Indoor Play Area,Food & Drink Shop,Electronics Store,Hotel
6,Merton,Pub,Park,Coffee Shop,Bar,Sushi Restaurant
7,Redbridge,Pub,Park,Coffee Shop,Café,Restaurant


In [25]:
brgh_grouped_clustering = Brgh_grouped.drop('Borough', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(brgh_grouped_clustering)
kmeans.labels_[0:10]

array([2, 2, 2, 2, 0, 1, 0, 0], dtype=int32)

Join all of our dataframes together.

In [28]:
brgh_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Borough_merged = pd.merge(fin, brgh_venues_sorted, on='Borough')
Borough_merged

,Borough,Area,Population,Latitude,Longitude,Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,194352,51.554117,0.150504,102.25,2,Park,Supermarket,Coffee Shop,Grocery Store,Pub
1,Bexley,23.38,236687,39.969238,-82.936864,97.00,2,Pizza Place,Coffee Shop,Ice Cream Shop,Discount Store,Restaurant
2,Bromley,57.97,317899,51.402805,0.014814,118.50,2,Coffee Shop,Grocery Store,Pub,Park,Gym / Fitness Center
3,Enfield,31.74,320524,51.652085,-0.081018,102.25,2,Coffee Shop,Pub,Turkish Restaurant,Café,Park
4,Haringey,11.42,263386,51.587930,-0.105410,107.75,0,Pub,Café,Turkish Restaurant,Park,Coffee Shop
5,Havering,43.35,242080,51.004361,-2.337475,86.00,1,IT Services,Indoor Play Area,Food & Drink Shop,Electronics Store,Hotel
6,Merton,14.52,203223,51.410803,-0.188099,123.75,0,Pub,Park,Coffee Shop,Bar,Sushi Restaurant
7,Redbridge,21.78,288272,51.576320,0.045410,118.50,0,Pub,Park,Coffee Shop,Café,Restaurant


In [29]:
map_cluster = folium.Map(location=[London_latitude, London_longitude], zoom_start=10)
kclusters = 3
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster, rent, pop in zip(Borough_merged['Latitude'], Borough_merged['Longitude'], Borough_merged['Borough'], Borough_merged['Cluster Label'], Borough_merged['Max_Rent'], Borough_merged['Population']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + " " + "Rent " + str(rent) + " " + "Population " + str(pop), parse_html=True)
    folium.CircleMarker([lat, lon],
                       radius = 25,
                       popup = label,
                       color = rainbow[cluster-1],
                       fill = True,
                       fill_color = rainbow[cluster-1],
                       fill_opacity = 0.7).add_to(map_cluster)
    
map_cluster

More thorough look into each one with a bit more detail, to see, which one is better suited for opening a venue.

In [31]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 0, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Haringey,11.42,263386,-0.105410,0,Pub,Café,Turkish Restaurant,Park,Coffee Shop
6,Merton,14.52,203223,-0.188099,0,Pub,Park,Coffee Shop,Bar,Sushi Restaurant
7,Redbridge,21.78,288272,0.045410,0,Pub,Park,Coffee Shop,Café,Restaurant


In [32]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 1, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Havering,43.35,242080,-2.337475,1,IT Services,Indoor Play Area,Food & Drink Shop,Electronics Store,Hotel


In [33]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 2, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,194352,0.150504,2,Park,Supermarket,Coffee Shop,Grocery Store,Pub
1,Bexley,23.38,236687,-82.936864,2,Pizza Place,Coffee Shop,Ice Cream Shop,Discount Store,Restaurant
2,Bromley,57.97,317899,0.014814,2,Coffee Shop,Grocery Store,Pub,Park,Gym / Fitness Center
3,Enfield,31.74,320524,-0.081018,2,Coffee Shop,Pub,Turkish Restaurant,Café,Park


<h3>Results</h3>

Results of the above analysis and clustering cam be summarized:

<ol>
<li>The most popular social venues, ouside of Inner London boroughs are Pubs and Coffee shops</li>
<li>Northern boroughs are more prone to visiting pubs, whereas southern boroughs are most likely to shop and have the social life from home<li>
<li>Within top 5 places of interest in every borough is an ethnic restaurant
<li>Rent price is not so much a factor for going out - the demand is not affected by difference in costs
</ol>